In [1]:
# BRCF..ipynb 
import numpy as np
import pandas as pd
from math import sqrt

In [2]:
"""
Marcel Caraciolo suggested a collaborative filtering algorithm simply using distance-base similarity score
[http://aimotion.blogspot.com/2009/11/collaborative-filtering-implementation.html].  

We tried his approach on Book-Crossing Dataset [http://www2.informatik.uni-freiburg.de/~cziegler/BX/].

Since this dataset has many zero implicit ratings, we replaced these ratings with average ratings if available.
Then we also tried his approach with our enhanced dataset to examine how more ratings impact collaborative filtering.
"""

'\nMarcel Caraciolo suggested a collaborative filtering algorithm simply using distance-base similarity score\n[http://aimotion.blogspot.com/2009/11/collaborative-filtering-implementation.html].  \n\nWe tried his approach on Book-Crossing Dataset [http://www2.informatik.uni-freiburg.de/~cziegler/BX/].\n\nSince this dataset has many zero implicit ratings, we replaced these ratings with average ratings if available.\nThen we also tried his approach with our enhanced dataset to examine how more ratings impact collaborative filtering.\n'

In [ ]:
"""
Data Structures

We use two types of 2D matrices, implemented using Python dictionary, to capture the user-item-rating information.

1. prefs
* In order to provide recommendation for a user, 2D prefs marix will have users as rows and items as columns.
** i.e., rating stores as prefs[user][item] 
* We provide two prefs matrices:
1a. prefsLess which is based on non-zero ratings from original dataset.
1b. prefsMore which includes prefsLess plus additional non-zero ratings by imputing average ratings. 

2. critics
* In order to provide recommendation for an item, 2D critics marix will have items as rows and users as columns.
** i.e., rating stores as critics[item][user] 
* We provide two critics matrices:
2a. criticsLess which is a re-arrangement of prefsLess.  
2b. criticsMore which is a re-arrangement of prefsMore.  

"""

In [3]:
"""
* Build prefsLess using BX-Books.csv and BX-Book-Ratings.csv.
* Both files are from original Book-Crossing Dataset.  
* They use ";" as field separator.
"""
def buildPrefsLess():
	#Load books information
	books = {}
	for line in open("data/BX-Books.csv"):
		line = line.replace('"', "")
		(id,title) = line.split(";")[0:2]
		books[id] = title
	
	#Load ratings information
	prefs = {}
	countValueError = 0
	countKeyError = 0
	for line in open("data/BX-Book-Ratings.csv"):
		line = line.replace('"', "")
		line = line.replace("\\","")
		(user,bookid,rating) = line.split(";")[0:3]
		try:
			if float(rating) > 0.0:
				prefs.setdefault(user,{})
				prefs[user][books[bookid]] = float(rating)
		except ValueError:
			countValueError += 1
		except KeyError:
			countKeyError += 1
	print "# .Value Error : " + str(countValueError)
	print "# ...Key Error : " + str(countKeyError)
	return prefs

In [4]:
prefsLess = buildPrefsLess()

# .Value Error : 1
# ...Key Error : 49818


In [5]:
"""
* Build prefsMore using MC.Books.csv and Good.Ratings.csv.
* They are based on BX-Books.csv and BX-Book-Ratings.csv, 
  with some data cleaning and zero-ratings replacement (with average ratings) when available.  
* They use "," as field separator.
"""
def buildPrefsMore():
	#Load books information
	books = {}
	for line in open("data/MC.Books.csv"):
		line = line.replace('"', "")
		line = line.replace("\\","")
		line = line.replace("\n","")                
		(id,title) = line.split(",")[0:2]
		books[id] = title
	
	#Load ratings information
	prefs = {}
	countValueError = 0
	countKeyError = 0
	for line in open("data/Good.Ratings.csv"):
		line = line.replace('"', "")
		line = line.replace("\\","")
		line = line.replace("\n","")        
		(user,bookid,rating) = line.split(",")[0:3]
		try:
			if float(rating) > 0.0:
				prefs.setdefault(user,{})
				prefs[user][books[bookid]] = float(rating)
		except ValueError:
			countValueError += 1
		except KeyError:
			countKeyError += 1
	print "# .Value Error : " + str(countValueError)
	print "# ...Key Error : " + str(countKeyError)
	return prefs

In [6]:
prefsMore = buildPrefsMore()

# .Value Error : 2
# ...Key Error : 76127


In [7]:
print "# prefsLess values : " + str(sum(len(v) for v in prefsLess.itervalues()))
print "# prefsLess values : " + str(sum(len(v) for v in prefsMore.itervalues()))

# prefsLess values : 382807
# prefsLess values : 846584


In [8]:
"""
Spot check.  Same.
"""
print "prefsLess['98556'] : ", prefsLess['98556']
print "prefsMore['98556'] : ", prefsMore['98556']

prefsLess['98556'] :  {'Foundation (Foundation Novels (Paperback))': 7.0}
prefsMore['98556'] :  {'Foundation (Foundation Novels (Paperback))': 7.0}


In [9]:
"""
Spot check. Same.
"""
print "prefsLess['180727'] : ", prefsLess['180727']
print "prefsMore['180727'] : ", prefsMore['180727']

prefsLess['180727'] :  {'Foundation (Foundation Novels (Paperback))': 3.0, 'Brave New World': 6.0, 'Jeeves in the morning (Perennial library)': 7.0, 'Foundation and Empire (Foundation Novels (Paperback))': 4.0, "Foundation's Edge : The Foundation Novels (Foundation Novels (Paperback))": 3.0, 'Second Foundation (Foundation Novels (Paperback))': 4.0}
prefsMore['180727'] :  {'Foundation (Foundation Novels (Paperback))': 3.0, 'Brave New World': 6.0, 'Jeeves in the morning (Perennial library)': 7.0, 'Foundation and Empire (Foundation Novels (Paperback))': 4.0, "Foundation's Edge : The Foundation Novels (Foundation Novels (Paperback))": 3.0, 'Second Foundation (Foundation Novels (Paperback))': 4.0}


In [10]:
"""
Spot check. prefsMore has user 276725, but not found in prefsLess.

In "BX-Book-Ratings.csv", user "276725" has only 1 entry and with zero rating:
> bc7_cwang@nodeD:~/Book.Recommendation.CF$ grep '"276725"' data/BX-Book-Ratings.csv 
> "276725";"034545104X";"0" # According to Amazon, <ISBN-10: 034545104X> is for hardcover <Flesh Tones>.
Since "Good.Ratings.csv" has average rating 6 for <Flesh Tones>, prefsMore capture the rating.
"""
print prefsLess.has_key('276725')
print prefsMore.has_key('276725')
print "prefsMore['276725'] : ", prefsMore['276725']

False
True
prefsMore['276725'] :  {'Flesh Tones: A Novel': 6.0}


In [11]:
"""
Function transform from prefs[user][item] to critics[item][user].
"""
def transformPrefs(prefs):
	results = {}
	for user in prefs:
		for item in prefs[user]:
			results.setdefault(item,{})
			#Flip item and user
			results[item][user] = prefs[user][item]
	return results

In [12]:
criticsLess = transformPrefs(prefsLess)
criticsMore = transformPrefs(prefsMore)

In [13]:
print "# criticsLess values : " + str(sum(len(v) for v in criticsLess.itervalues()))
print "# criticsLess values : " + str(sum(len(v) for v in criticsMore.itervalues()))

# criticsLess values : 382807
# criticsLess values : 846584


In [14]:
"""
Functions calculate distance-base similarity score:
1. sim_euclidean : calculate the Euclidean distance between user1 and user2.
2. sim_pearson   : calculate the Pearson correlation coefficient for user1 and user2.
"""

# --- sim_euclidean --------------------

def sim_euclidean(prefs, user1, user2):
	#Get the list of shared_items
	si = {}
	for item in prefs[user1]:
		if item in prefs[user2]:
			si[item] = 1

	#if they have no rating in common, return 0
	if len(si) == 0: 
		return 0

	#Add up the squares of all differences
	sum_of_squares = sum([pow(prefs[user1][item]-prefs[user2][item],2) for item in prefs[user1] if item in prefs[user2]])

	return 1 / (1 + sum_of_squares)

# --- sim_pearson --------------------

def sim_pearson(prefs, user1, user2):
	#Get the list of mutually rated items
	si = {}
	for item in prefs[user1]:
		if item in prefs[user2]: 
			si[item] = 1

	#if they are no rating in common, return 0
	if len(si) == 0:
		return 0

	#sum calculations
	n = len(si)

	#sum of all preferences
	sum1 = sum([prefs[user1][it] for it in si])
	sum2 = sum([prefs[user2][it] for it in si])

	#Sum of the squares
	sum1Sq = sum([pow(prefs[user1][it],2) for it in si])
	sum2Sq = sum([pow(prefs[user2][it],2) for it in si])

	#Sum of the products
	pSum = sum([prefs[user1][it] * prefs[user2][it] for it in si])

	#Calculate r (Pearson score)
	num = pSum - (sum1 * sum2/n)
	den = sqrt((sum1Sq - pow(sum1,2)/n) * (sum2Sq - pow(sum2,2)/n))
	if den == 0:
		return 0

	r = num/den

	return r

In [15]:
"""
Returns the best matches for user from the prefs dictionary
Number of the results and similiraty function are optional params.
"""

def topMatches(prefs, user, n=5, similarity=sim_pearson):
	scores = [(similarity(prefs,user,other),other)
				for other in prefs if other != user]
	scores.sort()
	scores.reverse()
	return scores[0:n]

In [16]:
"""
Gets recommendations for a user by using a weighted average
of every other user's rankings
"""

def getRecommendations(prefs, user, similarity=sim_pearson):
	totals = {}
	simSums = {}

	for other in prefs:
		#don't compare me to myself
		if other == user:
			continue
		sim = similarity(prefs,user,other)

		#ignore scores of zero or lower
		if sim <= 0: 
			continue
		for item in prefs[other]:
			#only score books i haven't seen yet
			if item not in prefs[user] or prefs[user][item] == 0:
				#Similarity * score
				totals.setdefault(item,0)
				totals[item] += prefs[other][item] * sim
				#Sum of similarities
				simSums.setdefault(item,0)
				simSums[item] += sim

	#Create the normalized list
	rankings = [(total/simSums[item],item) for item,total in totals.items()]

	#Return the sorted list
	rankings.sort()
	rankings.reverse()
	return rankings

In [17]:
"""
Let's try to find top 10 users like user 177432 and 180727.
* Case-1 : user 177432
** Top 10 selected by euclidean are different than pearson.
** Top 10 selected by euclidean are different when available number of ratings are different,
   although all selected scores are the same (1.0).
** Top 10 selected by pearson are different when available number of ratings are different.
   When more ratings areavailable, pearson select more higher scores candidates.    
* Case-2 : user 180727
** Top 10 selected by euclidean are different than pearson.
** Top 10 selected by euclidean are the same  number.
** Top 10 selected by pearson are different when available number of ratings are different.
   When less ratings areavailable, pearson select only 3 candidates with non-zero scores.
   When more ratings areavailable, pearson select all 10 candidates with non-zero scores. 
"""
print "<<topMatches(prefsLess, '177432', 10, sim_euclidean)>>"
print topMatches(prefsLess, '177432', 10, sim_euclidean)
print "<<topMatches(prefsMore, '177432', 10, sim_euclidean)>>"
print topMatches(prefsMore, '177432', 10, sim_euclidean)
print "<<topMatches(prefsLess, '177432', 10, sim_pearson)>>"
print topMatches(prefsLess, '177432', 10, sim_pearson)
print "<<topMatches(prefsMore, '177432', 10, sim_pearson)>>"
print topMatches(prefsMore, '177432', 10, sim_pearson)
print " "
print "<<topMatches(prefsLess, '180727', 10, sim_euclidean)>>"
print topMatches(prefsLess, '180727', 10, sim_euclidean)
print "<<topMatches(prefsMore, '180727', 10, sim_euclidean)>>"
print topMatches(prefsMore, '180727', 10, sim_euclidean)
print "<<topMatches(prefsLess, '180727', 10, sim_pearson)>>"
print topMatches(prefsLess, '180727', 10, sim_pearson)
print "<<topMatches(prefsMore, '180727', 10, sim_pearson)>>"
print topMatches(prefsMore, '180727', 10, sim_pearson)

<<topMatches(prefsLess, '177432', 10, sim_euclidean)>>
[(1.0, '98575'), (1.0, '98484'), (1.0, '98230'), (1.0, '97968'), (1.0, '97721'), (1.0, '96463'), (1.0, '96440'), (1.0, '95567'), (1.0, '95173'), (1.0, '9502')]
<<topMatches(prefsMore, '177432', 10, sim_euclidean)>>
[(1.0, '99630'), (1.0, '99051'), (1.0, '98686'), (1.0, '98618'), (1.0, '98575'), (1.0, '98484'), (1.0, '98422'), (1.0, '98230'), (1.0, '98159'), (1.0, '97968')]
<<topMatches(prefsLess, '177432', 10, sim_pearson)>>
[(1.0000000000000107, '199515'), (1.0000000000000107, '110112'), (1.000000000000007, '94923'), (1.000000000000007, '51883'), (1.000000000000007, '259320'), (1.0000000000000033, '89602'), (1.0000000000000018, '94307'), (1.0, '99766'), (1.0, '98026'), (1.0, '97324')]
<<topMatches(prefsMore, '177432', 10, sim_pearson)>>
[(1.000000000000016, '69554'), (1.000000000000016, '62755'), (1.000000000000016, '45557'), (1.000000000000016, '38464'), (1.000000000000016, '237154'), (1.000000000000016, '222716'), (1.00000000000

In [18]:
"""
Let's try to recommend 5 books to user 177432 and 180727.
* Recommendations are all different.
"""
print "<<getRecommendations(prefsLess, '177432', sim_euclidean)[0:5]>>"
print getRecommendations(prefsLess, '177432', sim_euclidean)[0:5]
print "<<getRecommendations(prefsMore, '177432', sim_euclidean)[0:5]>>"
print getRecommendations(prefsMore, '177432', sim_euclidean)[0:5]
print "<<getRecommendations(prefsLess, '177432', sim_pearson)[0:5]>>"
print getRecommendations(prefsLess, '177432', sim_pearson)[0:5]
print "<<getRecommendations(prefsMore, '177432', sim_pearson)[0:5]>>"
print getRecommendations(prefsMore, '177432', sim_pearson)[0:5]
print " "
print "<<getRecommendations(prefsLess, '180727', sim_euclidean)[0:5]>>"
print getRecommendations(prefsLess, '180727', sim_euclidean)[0:5]
print "<<getRecommendations(prefsMore, '180727', sim_euclidean)[0:5]>>"
print getRecommendations(prefsMore, '180727', sim_euclidean)[0:5]
print "<<getRecommendations(prefsLess, '180727', sim_pearson)[0:5]>>"
print getRecommendations(prefsLess, '180727', sim_pearson)[0:5]
print "<<getRecommendations(prefsMore, '180727', sim_pearson)[0:5]>>"
print getRecommendations(prefsMore, '180727', sim_pearson)[0:5]

<<getRecommendations(prefsLess, '177432', sim_euclidean)[0:5]>>
[(10.000000000000002, 'Zaftig: The Case for Curves'), (10.000000000000002, "You're My Baby (9 Months Later) (Harlequin Superromance, No. 1059)"), (10.000000000000002, "Yachtsman's Eight Language Dictionary: English, French, German, Dutch, Danish, Italian, Spanish, Portuguese"), (10.000000000000002, 'World of Robert Bateman'), (10.000000000000002, 'Witchdame')]
<<getRecommendations(prefsMore, '177432', sim_euclidean)[0:5]>>
[(10.000000000000004, 'The Magic School Bus and the Electric Field Trip'), (10.000000000000004, 'The Drowned and the Saved'), (10.000000000000004, 'Southern Belle (Mira)'), (10.000000000000004, 'Sorrow Floats'), (10.000000000000004, 'Red Branch')]
<<getRecommendations(prefsLess, '177432', sim_pearson)[0:5]>>
[(10.000000000000002, "Uncle John's Unstoppable Bathroom Reader (Bathroom Reader Series)"), (10.000000000000002, 'Twelve Steps and Twelve Traditions'), (10.000000000000002, 'Traveling Light: Releasin

In [19]:
"""
If you give function topMatches a prefs matrix and an user as input, it returns top matched users.

If you give function topMatches a critics matrix and a book title as input, it returns similar books.

Let's try to find 5 similar books as 'Drums of Autumn'.
* Only "Zukunftsmarkt Business on Demand." appears twice using euclidean distance.
"""
print "<<topMatches(criticsLess, 'Drums of Autumn', 5, sim_euclidean)>>"
print topMatches(criticsLess, 'Drums of Autumn', 5, sim_euclidean)
print "<<topMatches(criticsMore, 'Drums of Autumn', 5, sim_euclidean)>>"
print topMatches(criticsMore, 'Drums of Autumn', 5, sim_euclidean)
print "<<topMatches(criticsLess, 'Drums of Autumn', 5, sim_pearson)>>"
print topMatches(criticsLess, 'Drums of Autumn', 5, sim_pearson)
print "<<topMatches(criticsMore, 'Drums of Autumn', 5, sim_pearson)>>"
print topMatches(criticsMore, 'Drums of Autumn', 5, sim_pearson)

<<topMatches(criticsLess, 'Drums of Autumn', 5, sim_euclidean)>>
[(1.0, '\\Good Housekeeping\\ Soups and Starters'), (1.0, "\\A Room of One's Own\\, and \\Three Guineas\\ (Oxford World's Classics)"), (1.0, 'Zukunftsmarkt Business on Demand. Unternehmenserfolg durch st\xc3?\xc2\xa4ndige Verf\xc3?\xc2\xbcgbarkeit.'), (1.0, 'Zoya'), (1.0, "Zest: Cosmopolitan's Health and Beauty Handbook")]
<<topMatches(criticsMore, 'Drums of Autumn', 5, sim_euclidean)>>
[(1.0, 'christmas on snowbird mountain'), (1.0, 'bridegroom on her doorstep  (white weddings)'), (1.0, 'Zuleika Dobson (Modern Library (Paperback))'), (1.0, 'Zukunftsmarkt Business on Demand. Unternehmenserfolg durch st\xc3?\xc2\xa4ndige Verf\xc3?\xc2\xbcgbarkeit.'), (1.0, 'ZigZag: A Novel')]
<<topMatches(criticsLess, 'Drums of Autumn', 5, sim_pearson)>>
[(1.0, 'Year of Wonders'), (1.0, 'Velvet Angel'), (1.0, 'Twice Loved'), (1.0, 'Trying to Save Piggy Sneed'), (1.0, 'The Zebra Wall')]
<<topMatches(criticsMore, 'Drums of Autumn', 5, sim_pe

In [21]:
"""
If you give function getRecommendations a prefs matrix and an user as input, it returns few book recommendations.

If you give function getRecommendations a critics matrix and a book title as input, it returns few users who may 
want to read the book.

Let's try to find 5 users who may want to read 'The Weight of Water'.
* Recommendations are all different.
"""
print "<<getRecommendations(criticsLess, 'The Weight of Water', sim_euclidean)[0:5]>>"
print getRecommendations(criticsLess, 'The Weight of Water', sim_euclidean)[0:5]
print "<<getRecommendations(criticsMore, 'The Weight of Water', sim_euclidean)[0:5]>>"
print getRecommendations(criticsMore, 'The Weight of Water', sim_euclidean)[0:5]
print "<<getRecommendations(criticsLess, 'The Weight of Water', sim_pearson)[0:5]>>"
print getRecommendations(criticsLess, 'The Weight of Water', sim_pearson)[0:5]
print "<<getRecommendations(criticsMore, 'The Weight of Water', sim_pearson)[0:5]>>"
print getRecommendations(criticsMore, 'The Weight of Water', sim_pearson)[0:5]

<<getRecommendations(criticsLess, 'The Weight of Water', sim_euclidean)[0:5]>>
[(10.000000000000002, '99085'), (10.000000000000002, '96635'), (10.000000000000002, '95375'), (10.000000000000002, '94025'), (10.000000000000002, '9219')]
<<getRecommendations(criticsMore, 'The Weight of Water', sim_euclidean)[0:5]>>
[(10.000000000000002, '98735'), (10.000000000000002, '97921'), (10.000000000000002, '97421'), (10.000000000000002, '97139'), (10.000000000000002, '94125')]
<<getRecommendations(criticsLess, 'The Weight of Water', sim_pearson)[0:5]>>
[(10.000000000000002, '92048'), (10.000000000000002, '211152'), (10.000000000000002, '198996'), (10.000000000000002, '156467'), (10.0, '99298')]
<<getRecommendations(criticsMore, 'The Weight of Water', sim_pearson)[0:5]>>
[(10.000000000000002, '99298'), (10.000000000000002, '98344'), (10.000000000000002, '78631'), (10.000000000000002, '43937'), (10.000000000000002, '41343')]


In [ ]:
"""
Although Marcel Caraciolo's collaborative filtering algorithm is simple, it provide us many functionalities:
* If you give function topMatches a prefs matrix and an user as input, it returns top matched users.
* If you give function topMatches a critics matrix and a book title as input, it returns similar books.
* If you give function getRecommendations a prefs matrix and an user as input, it returns few book recommendations.
* If you give function getRecommendations a critics matrix and a book title as input, it returns few users who may 
want to read the book.

However, it is difficult to jsutify if any recommendations are valid since:
* The dataset is not ideally clean.
* The dataset does not have enough information about users or books.
* Marcel Caraciolo's approach does not make use of users' profile.
"""